In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import os
import zipfile
from io import BytesIO, StringIO

from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

In [ ]:
try:
    raw_df = pd.read_csv('/content/drive/MyDrive/Wikipedia_movie_meta_data.csv')
    display(raw_df.sample(5))
except FileNotFoundError:
    print('Wikipedia_movie_meta_data.csv not found. Please upload it to MyDrive')
print(raw_df.shape)

,Age Restrict,Akas,Awards,Budget,Cast,Cast1,Casting Directors,Countries,Director,Directors,...,Title1,Wiki Page,Writers,Year,Count Matches,F1,Imdb User Rating,Imdbid,Metascore,Number Of Imdb User Votes
429,"Australia:MA15+, Austria:14, Canada:14A::(Albe...",The Happy Prince - L'ultimo ritratto di Oscar ...,"Bavarian Film Award 2018, Teddy 2018, British ...",NaN,"Colin Firth, Colin Morgan, Emily Watson, Ruper...",NaN,Celestia Fox,"United Kingdom, Belgium, Italy, Germany",NaN,Rupert Everett,...,NaN,NaN,Rupert Everett,2018,0,425.0,6,2404639,64,4397
401,"Argentina:13, Australia:M, Belgium:KT/EA, Braz...","The Haunting of Hill House (United States), Th...","Blockbuster Entertainment Award 2000, BMI Film...","$80,000,000 (estimated)","Liam Neeson, Catherine Zeta-Jones, Owen Wilson...","Liam Neeson, Catherine Zeta-Jones, Lili Taylor...",Randi Hiller,"United States, United Kingdom",Jan de Bont,Jan de Bont,...,The Haunting,https://en.wikipedia.org/wiki/The_Haunting_(19...,"David Self, Shirley Jackson",1999,1,398.0,5,171363,42,71249
1437,"Argentina:13, Australia:M, Canada:13+::(Quebec...","King of the Kickboxers 2 (United States), King...",NaN,NaN,"Reese Madigan, Trent Bushey, Daniel Dae Kim, B...",NaN,NaN,"United States, Hong Kong",Lucas Lo,Lucas Lowe,...,American Shaolin,https://en.wikipedia.org/wiki/American_Shaolin,Keith W. Strandberg,1991,1,2090.0,5,101327,-1,1288
1339,"Argentina:13, Australia:M, Austria:12, Brazil:...",The Girl with the Red Riding Hood (United Stat...,"CDG Award 2012, HFCS Award 2011, Young Artist ...","$42,000,000 (estimated)","Amanda Seyfried, Gary Oldman, Billy Burke, Shi...","Amanda Seyfried, Gary Oldman, Billy Burke, Shi...",Ronna Kress,"United States, Canada",Catherine Hardwicke,Catherine Hardwicke,...,Red Riding Hood,https://en.wikipedia.org/wiki/Red_Riding_Hood_...,David Leslie Johnson-McGoldrick,2011,1,1877.0,5,1486185,29,107520
683,"Argentina:Atp, Finland:S, Iceland:L, Japan:G, ...","L'odyssée de Charles Lindbergh (France), Lindb...","Oscar 1958, Boxoffice Blue Ribbon Award 1957, ...",NaN,"James Stewart, Murray Hamilton, Patricia Smith...",James Stewart,NaN,United States,Billy Wilder,Billy Wilder,...,The Spirit of St. Louis,https://en.wikipedia.org/wiki/The_Spirit_of_St...,"Charles A. Lindbergh, Billy Wilder, Wendell Ma...",1957,1,677.0,7,51003,-1,7280


(2901, 38)


In [ ]:
# clean the column names and only keep the column I need
df = raw_df.copy(deep=True)
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2901 entries, 0 to 2900
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age_restrict               2569 non-null   object 
 1   akas                       2694 non-null   object 
 2   awards                     2285 non-null   object 
 3   budget                     1657 non-null   object 
 4   cast                       2849 non-null   object 
 5   cast1                      1869 non-null   object 
 6   casting_directors          2183 non-null   object 
 7   countries                  2720 non-null   object 
 8   director                   1877 non-null   object 
 9   directors                  2701 non-null   object 
 10  filename                   2889 non-null   object 
 11  genre                      1877 non-null   object 
 12  genres                     2889 non-null   object 
 13  imdb_id                    2889 non-null   float

In [ ]:
df = df[['imdbid','title','screenplay', 'plot','plot_outline','synopsis']]
print(df.info())
df.sample(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2901 entries, 0 to 2900
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   imdbid        2901 non-null   int64 
 1   title         2901 non-null   object
 2   screenplay    2889 non-null   object
 3   plot          2752 non-null   object
 4   plot_outline  2498 non-null   object
 5   synopsis      1891 non-null   object
dtypes: int64(1), object(5)
memory usage: 136.1+ KB
None


,imdbid,title,screenplay,plot,plot_outline,synopsis
604,817228,The Tattooist,the tattooist by Matthew grainger & Jonathan K...,A young artist unknowingly plays a role in rel...,American tattoo artist Jake Sawyer wanders the...,NaN
786,3099498,Tusk,tusk by Kevin Smith base on a story from smodc...,A brash and arrogant podcaster gets more than ...,When podcaster Wallace travels to Canada to in...,Wallace Bryton (Justin Long) and friend Teddy ...
182,58946,The Battle of Algiers,the Battle of algier script by gillo pontecorv...,"In the 1950s, fear and violence escalate as th...",A film commissioned by the Algerian government...,The opening scene is of a man who has presumab...
2203,49513,Moby Dick,moby dick part one act 1 title over : ext . na...,The sole survivor of a lost whaling ship relat...,This classic story by Herman Melville revolves...,MOBY DICKA man is seen walking along hilly ter...
2455,47443,Salt of the Earth,Salt of the Earth screenplay by Michael Wilson...,Mexican workers at a Zinc mine call a general ...,Based on an actual strike against the Empire Z...,NaN
2207,74695,Cross of Iron,Cross of Iron ( 1977 ) by Sam peckinpah more i...,German commander Captain Hauptmann Stransky (M...,"In 1943, on the Russian front, decorated leade...",NaN
1603,73349,Mandingo,mandingo from the novel by Kyle onstott second...,An 1840s slaveowner trains one of his slaves t...,It's the mid-nineteenth century Louisiana. Fal...,NaN
1554,68230,Asylum,asylum ext - prague STREET - night a luxury po...,In order to secure a job at a mental instituti...,A young psychiatrist interviews four inmates i...,The young Dr. Martin goes to a job interview a...
2130,1022603,500 Days of Summer,500 day of summer written by Scott neustadter ...,An offbeat romantic comedy about a woman who d...,After it looks as if she's left his life for g...,(500) Days of Summer is presented in a non-chr...
1659,211933,Awake,written by Kyle killen teaser ext . hillside -...,A wealthy young man undergoing heart transplan...,"While undergoing open heart surgery, a man's f...",The Tagline is a gross misrepresentation of st...


In [ ]:
df['plot_len'] = df['plot'].str.len()
df['plot_outline_len'] = df['plot_outline'].str.len()
df['synopsis_len'] = df['synopsis'].str.len()
df.head()

,imdbid,title,screenplay,plot,plot_outline,synopsis,plot_len,plot_outline_len,synopsis_len
0,316654,Spider-Man 2,spider-man 2 written by Michael chabon fade in...,Peter Parker is beset with troubles in his fai...,Peter Parker is an unhappy man: after two year...,Peter Parker is finding his double life increa...,3201.0,706.0,6243.0
1,164167,Sidewalks of New York,the sidewalks of New York ( 1923 ) Art title -...,NaN,NaN,NaN,NaN,NaN,NaN
2,2761612,Short Term Pain Long Term Gain,short term 12 by destin Daniel cretton August ...,NaN,NaN,NaN,NaN,NaN,NaN
3,9106126,Shining The Light,""" the shining . "" Post production script . a s...",Shining The Light will have a panel of six you...,Shining The Light will have a panel of six you...,NaN,691.0,677.0,NaN
4,614236,Smoke Gets in Your Eyes,mad men written by Matthew Weiner one-hour Pil...,A woman who was assaulted and left blind leave...,A woman who was assaulted and left blind leave...,NaN,261.0,242.0,NaN


In [ ]:
final_df = df[['plot_outline','screenplay']]
final_df = final_df[~final_df['plot_outline'].isnull()]
final_df = final_df.rename(columns={'screenplay':'prompt'
                                  , 'plot_outline':'completion'})
print(final_df.shape)



(2498, 2)


In [ ]:
final_df['prompt_f1k'] = final_df['prompt'].apply(lambda x: ' '.join(str(x).split()[:1000]))
final_df = final_df.drop('prompt', axis=1)
final_df = final_df.rename(columns={'prompt_f1k': 'prompt'})

final_df.sample(5)

,completion,prompt
2771,"In a distant, but not so unrealistic, future w...",wall * E sereenplay by Andrew stanton Jim rear...
2558,Because his wife left him and took his son wit...,""" leaving las vega "" by Mike figgis base on th..."
421,"Gary Hart, a U.S. senator from Colorado, is th...",the Front runner written by Matt bai & jay car...
2531,"In Canada, a writer visits the Indian storytel...",life of pi written by David magee base on the ...
1166,"July, 1995, the time is out of joint. Two teen...",S. darko written by Nathan Atkins second draft...


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
train_df, val_test_df = train_test_split(final_df, test_size=0.2, random_state=266)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=266)

# Print the shapes of the resulting dataframes
print('Training set shape:', train_df.shape)
print('Validation set shape:', val_df.shape)
print('Test set shape:', test_df.shape)

Training set shape: (1998, 2)
Validation set shape: (250, 2)
Test set shape: (250, 2)


### TextRank

https://www.kaggle.com/code/kunjan6902/text-summarization-textrank

1.   Tokenize words in each sentence
2.   Build a similarity matrix
3.   Run PageRank algorithm
4.   Extract top sentences



In [ ]:
movie_example = 0

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
'''
Step 1. Tokenize words in each sentence
'''

from nltk import sent_tokenize, word_tokenize

sentences = sent_tokenize(df['screenplay'][movie_example])
tokenized_sentences = [word_tokenize(sent) for sent in sentences]

NameError: ignored

In [ ]:
# number of sentences
print('sentence count: ', len(sentences))

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

Text Preprocessing

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
#Removing stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new


clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
print(len(sentence_vectors))

In [ ]:
'''
Step 2. Build a similarity matrix using cosine similarity
'''

# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
'''
Step 3. Run PageRank algorithm
'''

import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
len(ranked_sentences)

In [ ]:
# Extract top 100 sentences as the summary
for i in range(100):
  print(ranked_sentences[i][1])

In [ ]:
top_100_sentences_combined = ''
for i in range(100):
  top_100_sentences_combined += (ranked_sentences[i][1])

In [ ]:
top_100_sentences = []

for i in range(100):
  top_100_sentences.append(ranked_sentences[i][1])

In [ ]:
ranked_sentences

In [ ]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q textsum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

cnnmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)
cnntokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("google/pegasus-cnn_dailymail")

config

PegasusConfig {
  "_name_or_path": "google/pegasus-cnn_dailymail",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "length_penalty": 0.8,
  "max_length": 128,
  "max_position_embeddings": 1024,
  "min_length": 3

In [ ]:
cnninputs = cnntokenizer(top_100_sentences_combined, max_length=1024, truncation=True, return_tensors="tf")

NameError: ignored

In [ ]:
# Generate Summary

from pprint import pprint

summary_ids = cnnmodel.generate(cnninputs["input_ids"]
)

pprint(cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

In [ ]:
# load contents from wikipedia summary

try:
    wikipedia_summary = pd.read_csv('/content/drive/MyDrive/W266_Movie_Data/Wikipedia_Summary.csv')
    wikipedia_summary.sort_values(by=['Title'])
    display(wikipedia_summary.head())
except FileNotFoundError:
    print('wikipedia_summary not found')

In [ ]:
# Generate Summary
summary_ids = cnnmodel.generate(cnninputs["input_ids"],
    max_length=512,
    min_length=150,
    no_repeat_ngram_size=3,
    num_beams=4
                             )
candidate = cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(candidate[0], compact=True)

In [ ]:
print(candidate)

In [ ]:
rouge = evaluate.load('rouge')
predictions = candidate
references = [reference_text]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

NameError: ignored

## Movie Scripts Corpus Dataset from Kaggle
(this is our main dataset)

Here is a link to the [BERT_annotations folder](https://drive.google.com/drive/folders/1vK54OZIAAfVD9xV9znonMUgcb058wfOp?usp=share_link)

Here is a link to the [subset_BERT_annotations folder](https://drive.google.com/drive/folders/1KN0EAqo3FqJqhJS2wv9K5FICU0aBeD0R?usp=share_link)

### Import Movie Scripts

In [ ]:
# load contents from BERT annotations

bert_annotations_file_path = '/content/drive/MyDrive/W266_Movie_Data/subset_BERT_annotations/'

try:
    all_files = os.listdir(bert_annotations_file_path)
    print(all_files)
except FileNotFoundError:
    print('subset_BERT_annotations.zip not found')

['3 Idiots_1187043_anno.txt', '3 10 to Yuma_0381849_anno.txt', '3 Godfathers_0040064_anno.txt']


### Functions

In [ ]:
words = 'text: Rancho slices the blower into two cups.'
words.split(':')

['text', ' Rancho slices the blower into two cups.']

In [ ]:
def get_movie_title(script_txt_file):
    '''get the movie title without the unique identifier and _anno.txt suffix'''
    movie_title = script_txt_file.split('_')[0]
    return movie_title



def get_script_length(file_path, script_txt_file):
    '''calculate the number of lines in a BERT annotated script'''
    with open(str(file_path) + str(script_txt_file), 'r') as test_file:
        script_length = len(test_file.readlines())
    return script_length



def read_script(file_path, script_txt_file):
    '''read in the BERT annotated script'''
    script_text = open(str(file_path) + str(script_txt_file), 'r')
    # print(test_file.read())
    return script_text.read()



def count_script_elements(file_path, script_txt_file):
    '''count script elements such as dialog, text, speaker_heading, scene_heding'''
    script_element_dict = {}
    with open(str(file_path) + str(script_txt_file), 'r') as script_file:
        for line in script_file:
            script_element = line.split(':')[0]
            if script_element not in script_element_dict:
                script_element_dict[script_element] = 1
            else:
                script_element_dict[script_element] += 1
    return script_element_dict



def concatenate_script_element(file_path, script_txt_file, script_element):
    '''concatenate only a specific script element such as dialog, text, speaker_heading, scene_heding'''
    combined_script_element = ''
    with open(str(file_path) + str(script_txt_file), 'r') as script_file:
        for line in script_file:
            if line.split(':')[0].strip() == script_element:
                combined_script_element += line.split(':')[1]
            else:
              continue

    # remove empty lines
    text = os.linesep.join([s for s in combined_script_element.splitlines() if s])

    return text



def identify_characters(file_path, script_txt_file):
    '''count number of characters and their speaking parts'''
    speaker_heading_dict = {}
    with open(str(file_path) + str(script_txt_file), 'r') as script_file:
        for line in script_file:

            # if the script element is 'speaker_heading' then that is a character
            if 'speaker_heading' in line.split(':')[0]:
                # some speaker_headings do not contain character names
                if re.search('[a-zA-Z]', line.split(':')[1]) != None:

                    # remove leading and trailing spaces and trailing newlines
                    character = line.split(':')[1].lstrip().rstrip().replace('\n', '')

                    # remove text that is not uppercase
                    character = ''.join(ch for ch in character if not ch.islower())

                    # remove (O.S.) off screen from character name
                    character = character.replace(' (O.S.)', '')

                    # remove trailing punctuation
                    character = character.rstrip('.').rstrip('?').rstrip('!')

                    ##### NEED TO ADD LOGIC TO DEAL WITH CONTINUOUS, CONTINUED, and CONT'D #####

                    if character not in speaker_heading_dict:
                        speaker_heading_dict[character] = 1
                    else:
                        speaker_heading_dict[character] += 1

    # remove characters that only have one speaking line
    character_dict = {k:v for k, v in speaker_heading_dict.items() if v > 1}
    print(f'character_dict length before removing single speaking lines: {len(speaker_heading_dict)}')
    print(f'character_dict length after removing single speaking lines: {len(character_dict)}')

    return character_dict


# def check_if_english(file_path, script_txt_file, length_of_text_to_check):
#     '''check a portion of the script to see if it is english'''

#     # langid comes pre-trained on 97 languages
#     languages = ['af', 'am', 'an', 'ar', 'as', 'az', 'be', 'bg', 'bn', 'br',
#                  'bs', 'ca', 'cs', 'cy', 'da', 'de', 'dz', 'el', 'en', 'eo',
#                  'es', 'et', 'eu', 'fa', 'fi', 'fo', 'fr', 'ga', 'gl', 'gu',
#                  'he', 'hi', 'hr', 'ht', 'hu', 'hy', 'id', 'is', 'it', 'ja',
#                  'jv', 'ka', 'kk', 'km', 'kn', 'ko', 'ku', 'ky', 'la', 'lb',
#                  'lo', 'lt', 'lv', 'mg', 'mk', 'ml', 'mn', 'mr', 'ms', 'mt',
#                  'nb', 'ne', 'nl', 'nn', 'no', 'oc', 'or', 'pa', 'pl', 'ps',
#                  'pt', 'qu', 'ro', 'ru', 'rw', 'se', 'si', 'sk', 'sl', 'sq',
#                  'sr', 'sv', 'sw', 'ta', 'te', 'th', 'tl', 'tr', 'ug', 'uk',
#                  'ur', 'vi', 'vo', 'wa', 'xh', 'zh', 'zu']

#     # target languages
#     identifier = LanguageIdentifier.from_pickled_model(MODEL_FILE)
#     identifier.set_languages(languages)

#     # randomly choose start and end script lines based on length_of_text_to_check
#     # file lines start at index 0
#     script_start = random.randint(1, get_script_length(file_path, script_txt_file)) - 1
#     script_end = script_start + length_of_text_to_check - 1

#     # check only lines between start and end are english
#     with open(str(file_path) + str(script_txt_file), 'r') as script_file:
#         for i, line in enumerate(script_file):

#             text_to_check = ''

#             if i >= script_start and i <= script_end:

#                 # classifying by line is inconsistent, so append each line to a growing string
#                 line = line.split(':')[1].rstrip().replace('\n', '')
#                 text_to_check += line

#             elif i < script_start:
#                 continue

#             elif i > script_end:
#                 break

#             language, language_prob = identifier.classify(text_to_check)

#             # english code is 'en'
#             if language == 'en':
#                 is_english = True

#             else:
#                 is_english = False

#     return is_english

### Exploratory Data Analysis (on subset of data)

In [ ]:
# load contents from BERT annotations

bert_annotations_file_path = '/content/drive/MyDrive/W266_Movie_Data/subset_BERT_annotations/'

try:
    all_files = os.listdir(bert_annotations_file_path)
    print(all_files)
except FileNotFoundError:
    print('subset_BERT_annotations.zip not found')

['3 Idiots_1187043_anno.txt', '3 10 to Yuma_0381849_anno.txt', '3 Godfathers_0040064_anno.txt']


In [ ]:
# get script length of BERT annotated script
print(get_movie_title(all_files[0]), get_script_length(bert_annotations_file_path, all_files[0]))
print(get_movie_title(all_files[1]), get_script_length(bert_annotations_file_path, all_files[1]))
print(get_movie_title(all_files[2]), get_script_length(bert_annotations_file_path, all_files[2]))

3 Idiots 14828
3 10 to Yuma 4482
3 Godfathers 5748


In [ ]:
# read in one BERT annotation
script_text = read_script(bert_annotations_file_path, all_files[0])
print(script_text)

In [ ]:
# split the text file by the first : delimiter
# get the count of text, dialog, speaker, and scene_heading

print(get_movie_title(all_files[0]), count_script_elements(bert_annotations_file_path, all_files[0]))
print(get_movie_title(all_files[1]), count_script_elements(bert_annotations_file_path, all_files[1]))
print(get_movie_title(all_files[2]), count_script_elements(bert_annotations_file_path, all_files[2]))

3 Idiots {'dialog': 8537, 'text': 3622, 'speaker_heading': 2376, 'scene_heading': 293}
3 10 to Yuma {'dialog': 1648, 'scene_heading': 328, 'speaker_heading': 933, 'text': 1573}
3 Godfathers {'dialog': 2716, 'speaker_heading': 1021, 'scene_heading': 182, 'text': 1829}


In [ ]:
# identify characters and their speaking parts
print(get_movie_title(all_files[0]), identify_characters(bert_annotations_file_path, all_files[0]), '\n')
print(get_movie_title(all_files[1]), identify_characters(bert_annotations_file_path, all_files[1]), '\n')
print(get_movie_title(all_files[2]), identify_characters(bert_annotations_file_path, all_files[2]), '\n')

character_dict length before removing single speaking lines: 369
character_dict length after removing single speaking lines: 108
3 Idiots {'O B I': 3, 'AIR HOSTESS': 4, 'FARHAN': 301, 'LATER': 10, 'REM': 36, 'MOMENTS LATER': 3, 'DRIVER': 4, 'RAJU': 208, 'H': 28, 'RET': 28, 'T': 28, 'R': 3, 'CONTINUOUS': 55, 'CHATUR': 120, 'FLASHBACK': 4, 'MONTAGE': 5, 'REM (V.O.)': 5, 'FARHAN’S FATHER': 21, 'R (V.O.)': 5, '': 18, 'MILLIMETER': 23, 'NIGHT': 9, 'ALL STUDENTS': 2, ' (V.O.)': 24, 'RAGGING SENIOR 1': 24, '  1': 5, 'RAGGING SENIOR 2': 2, 'RANCHO': 350, 'U  1': 3, 'ICE -': 4, 'HEM (V.O.)': 6, 'ICE — CONTINUOUS': 8, 'DIRECTOR': 156, 'STE': 2, 'WH': 6, 'PROFESSOR POTDAR': 16, 'PROFESSOR': 6, '— DAY': 2, 'JY': 3, 'JOY’S FATHER': 3, 'JOY': 3, ' HT T': 2, 'U T   B': 2, 'ICE —': 3, 'TT': 3, 'PRIEST': 2, 'DAY': 5, 'FARHAN’S MOTHER': 3, 'RAJU’S MOTHER': 12, 'KAMMO': 3, 'RANCHO / FARHAN': 3, 'SUHAS': 27, 'AUNTY': 5, 'PIA': 143, 'BIST': 8, 'EM': 7, 'W': 3, 'ICE — NIGHT': 2, 'DUBEY': 11, 'MINISTER': 5, 

In [ ]:
'''
entire script
'''

# count number of lines
length_entire_script = script_text.count('\n')
print(length_entire_script)

# count number of words
num_words_entire_script = len(script_text.split())
print(num_words_entire_script)

14828
96897


In [ ]:
'''
dialog only
'''
dialog_only = concatenate_script_element(bert_annotations_file_path, all_files[0], 'dialog')
# print(get_movie_title(all_files[0]), concatenate_script_element(bert_annotations_file_path, all_files[0], 'dialog'), '\n')

# count number of lines
length_dialog_only = dialog_only.count('\n')
print(length_dialog_only)

# count number of words
num_words_dialog_only = len(dialog_only.split())
print(num_words_dialog_only)

8412
48878


In [ ]:
'''
text only
'''
text_only = concatenate_script_element(bert_annotations_file_path, all_files[0], 'text')
print(get_movie_title(all_files[0]), concatenate_script_element(bert_annotations_file_path, all_files[0], 'text'), '\n')

# count number of lines
length_text_only = text_only.count('\n')
print(length_text_only)

# count number of words
num_words_text_only = len(text_only.split())
print(num_words_text_only)

3 Idiots  Fits t THE ORIGINAL SCREENPLAY
 Fiems THE ORIGINAL SCREENPLAY
 poster on front cover by Rahul Nanda (H-One)
 ALL RIGHTS RESERVED. No part of
 There is no reading material available for film students or film lovers on the behind-
 is not just the screenplay; it is also a peek into the minds of its makers.
 book within 5 months of the release of the film. She has spent many long days
 and long nights chasing the cast and crew, writing and rewriting, with a smile on
 Kelkar for her eye for detail; Rohan Mapuskar for poring over the dialogues;
 Dileep Desai, Amit Gulati, Mustafa Neemuchwala, Teja Pratap, Insia Lacewalla for
 production support; Anupama Chopra for her invaluable insight.
 moved to India during Partition. Initially, they took shelter in refugee camps in
 in a bangle factory where he painted patterns on bangles. Later, he began selling
 ice-cream on the streets. Despite compelling and adverse circumstances, his thirst
 somehow managed to graduate. Fortified with a d

Since text only seems to be the most concise representation of the movie script, perform text rank on this giant string

#### Text Rank on text only data

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
'''
Step 1. Tokenize words in each sentence
'''

from nltk import sent_tokenize, word_tokenize

sentences = sent_tokenize(text_only)
tokenized_sentences = [word_tokenize(sent) for sent in sentences]

In [ ]:
# number of sentences
print('sentence count: ', len(sentences))

sentence count:  2249


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2023-03-28 02:00:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-03-28 02:00:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-03-28 02:00:10--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

400000

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

<ipython-input-40-57e05bf8eb2b>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [ ]:
#Removing stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new


clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
print(sentence_vectors)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
'''
Step 2. Build a similarity matrix using cosine similarity
'''

# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
'''
Step 3. Run PageRank algorithm
'''

import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
len(ranked_sentences)

2249

In [ ]:
# Extract top 100 sentences as the summary
for i in range(100):
  print(ranked_sentences[i][1])

The emotional high point for Rancho’s character is when he sits around
 waiting nervously for his friends to return from the situations (Farhan confronts
 his father and Raju goes for a job interview) they have gone to face, and the
 moment he realises that they have returned triumphant from these situations, he
 breaks down.
He skids on the urine-slick floor, steadying
 The Director sees three people run out
 As the other two run out of sight, Raju,
 with drunk bravado, fires a parting
 For just a moment, the randomly moving
 beam of the Director's torch catches Raju’s
 face, before his friends yank him away.
Perhaps the most telling example of this ‘solidification’ or ‘concretisation’ of a scene
 afier a long and eventful journey, the friends seem to reach Rancho, but when Farhan
 puts his hand on the man’ shoulder, the man who turns and looks at them is not their
 friend.
On
 the monitor is Joy's body, suspended from
 Even as time seems to stand still, Rancho,
 Farhan and Raju run t

In [ ]:
top_100_sentences_combined = ''
for i in range(100):
  top_100_sentences_combined += (ranked_sentences[i][1])

In [ ]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q textsum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

cnnmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)
cnntokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("google/pegasus-cnn_dailymail")

config

PegasusConfig {
  "_name_or_path": "google/pegasus-cnn_dailymail",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "length_penalty": 0.8,
  "max_length": 128,
  "max_position_embeddings": 1024,
  "min_length": 3

In [ ]:
cnninputs = cnntokenizer(top_100_sentences_combined, max_length=1024, truncation=True, return_tensors="tf")

In [ ]:
# Generate Summary

from pprint import pprint

summary_ids = cnnmodel.generate(cnninputs["input_ids"]
)

pprint(cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

/usr/local/lib/python3.9/dist-packages/transformers/generation/tf_utils.py:854: UserWarning: Using `max_length`'s default (128) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


('The emotional high point for Rancho’s character is when he sits around '
 'waiting nervously for his friends to return from the situations .<n>The same '
 'scene, without changing a single line of dialogue, with Rastogi sitting in '
 'an ordinary chair, would never have had the A laptop versus a letter .')


In [ ]:
# Generate Summary
summary_ids = cnnmodel.generate(cnninputs["input_ids"],
    max_length=512,
    min_length=150,
    no_repeat_ngram_size=3,
    num_beams=4
                             )
candidate = cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
pprint(candidate[0], compact=True)

('The emotional high point for Rancho’s character is when he sits around '
 'waiting nervously for his friends to return from the situations .<n>The same '
 'scene, without changing a single line of dialogue, with Rastogi sitting in '
 'an ordinary chair, would never have had the A laptop versus a letter — that '
 'is the essence of suppressing the helplessness of the retired man who breaks '
 'into an impromptu stripping act making the on-screen persona inseparable '
 'from the one in reality.<n>A young lady who hates her father’s archenemy and '
 'furiously confronts him must within the next three minutes break off her '
 'engagement with her fiancé, fall hopelessly in love with the same archenemy, '
 'and convincingly fantasise singing a rain-drenched love song with '
 'him.<n>Every object I listed here is now a part of what Rogers and '
 'Hammerstein have famously described as “a few of my favorite things.”')


In [ ]:
# load contents from wikipedia summary

try:
    wikipedia_summary = pd.read_csv('/content/drive/MyDrive/W266_Movie_Data/Wikipedia_Summary.csv')
    wikipedia_summary.sort_values(by=['Title'])
    display(wikipedia_summary.head())
except FileNotFoundError:
    print('wikipedia_summary not found')# load contents from wikipedia summary


# movie_name = all_files[0].split('_')[0]
# movie_script = (raw_text_lemmas_file_path + all_files[0])

# with open(movie_script, 'r') as file:
#     movie_script_data = file.read().rstrip()

reference_text = wikipedia_summary.loc[wikipedia_summary['Title'] == movie_name, 'Plot1']

,Title,Year,Plot1
0,3 Godfathers,1948,"Cattle rustlers Robert Hightower (John Wayne),..."
1,3 Idiots,2009,Onboard a departing Air India flight Farhan Qu...
2,3:10 to Yuma,2007,Dan Evans (Bale) is an impoverished rancher an...
3,8 Mile,2002,"In the Fall of 1995 in Detroit, Jimmy ""B-Rabbi..."
4,10 Cloverfield Lane,2016,"After breaking up with her boyfriend Ben, Mich..."


In [ ]:
import evaluate

rouge = evaluate.load('rouge')
predictions = candidate
references = [reference_text]
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.1773049645390071, 'rouge2': 0.021327014218009477, 'rougeL': 0.09456264775413711, 'rougeLsum': 0.13002364066193856}


In [ ]:
# language check

# stress test by looping multiple times to check random portions of the script
movie_count = {get_movie_title(all_files[0]): {True: 0, False: 0},
               get_movie_title(all_files[1]): {True: 0, False: 0},
               get_movie_title(all_files[2]): {True: 0, False: 0}}

for i in range(10):
    movie_count[get_movie_title(all_files[0])][check_if_english(bert_annotations_file_path, all_files[0], 500)] += 1
    movie_count[get_movie_title(all_files[1])][check_if_english(bert_annotations_file_path, all_files[1], 500)] += 1
    movie_count[get_movie_title(all_files[2])][check_if_english(bert_annotations_file_path, all_files[2], 500)] += 1
print(movie_count)

# print(get_movie_title(all_files[0]), check_if_english(bert_annotations_file_path, all_files[0], 500), '\n')
# print(get_movie_title(all_files[1]), check_if_english(bert_annotations_file_path, all_files[1], 500), '\n')
# print(get_movie_title(all_files[2]), check_if_english(bert_annotations_file_path, all_files[2], 250), '\n')

{'3 10 to Yuma': {True: 10, False: 0}, '3 Idiots': {True: 7, False: 3}, '3 Godfathers': {True: 9, False: 1}}


## Exploratory Data Analysis on entire dataset

In [ ]:
# load contents from BERT annotations

bert_annotations_file_path = '/content/drive/MyDrive/W266_Movie_Data/BERT_annotations/'

try:
    all_files_unsorted = os.listdir(bert_annotations_file_path)
    all_files = sorted(all_files_unsorted)
except FileNotFoundError:
    print('subset_BERT_annotations.zip not found')

In [ ]:
d = []
for i in range(len(all_files)):
    d.append(
        {
            'title': get_movie_title(all_files[i]),
            'length': get_script_length(bert_annotations_file_path, all_files[i])
        }
    )

pd.DataFrame(d)

,title,length
0,10 Cloverfield Lane,3314
1,10 Things I Hate About You,5969
2,12 Angry Men,5093
3,12 Monkeys,5049
4,12 Years a Slave,6069
...,...,...
1993,Your Highness,4799
1994,Zodiac,8390
1995,Zootopia,4304
1996,Zulu Dawn,3005


## Supplementary Title/Overview (Combined) Data from Wikipedia + TMDB

In [ ]:
# read in the combined_data.xslx
try:
    combined_data_df = pd.read_excel('/content/drive/MyDrive/W266_Movie_Data/combined_data.xlsx')
    display(combined_data_df.head())
except FileNotFoundError:
    print('combined_data.xlsx not found')

combined_data.xlsx not found


In [ ]:
# check the number of total samples
num_titles = len(combined_data_df['Title'].value_counts())
print(f'{num_titles} TOTAL movie titles')

# check the number of unique movie titles
num_unique_titles = len(combined_data_df['Title'].value_counts().loc[lambda x: x > 1])
print(f'{num_unique_titles} UNIQUE movie titles')

# check the number of duplicate movie titles
num_duplicate_titles = num_titles - num_unique_titles
print(f'{num_duplicate_titles} DUPLICATE movie titles')

In [ ]:
# there seems to be a type error when doing pandas methods on combined_data_df
def strip_character(dataCol):
    r = re.compile(r'[^a-zA-Z !@#$%&*_+-=|\:";<>,./()[\]{}\']')
    return r.sub('', str(dataCol))

combined_data_df['Title'] = combined_data_df['Title'].apply(lambda x: strip_character(x))